In [1]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("../zippedData/tn.movie_budgets.csv.gz")
df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [2]:
def string_to_int(string):
    return int(string.replace('$', '').replace(',',''))
df['worldwide_gross'] = df['worldwide_gross'].apply(lambda x: string_to_int(x))
df['production_budget'] = df['production_budget'].apply(lambda x: string_to_int(x))
df['domestic_gross'] = df['domestic_gross'].apply(lambda x: string_to_int(x))


In [16]:
df['profit'] = df['worldwide_gross'] - df['production_budget'] 
df['domestic_profit'] = df['domestic_gross'] - df['production_budget']
df['has foriegn'] = df['worldwide_gross'] > 0
df['roi'] = (df['profit']/ df['production_budget'])*100
df['domestic_roi'] = (df['domestic_profit']/ df['production_budget']) * 100

def get_iqr(df, col_name):
    hq = df[col_name].quantile(0.75) 
    median = df[col_name].quantile(0.5)
    lq = df[col_name].quantile(0.15)
    iqr = hq-lq
    return iqr, median

def is_outlier(x, iqr, median):
    if x > median + iqr*1.5:
        return True
    elif x < median - 1.5*iqr:
        return True
    else:
        return False

def find_outliers(df, column):
    """
    Returns a boolean series. True if outside IQR*1.5 range, False otherwise
    """
    IQR , median  = get_iqr(df, column)
    return df[column].apply(lambda x: is_outlier(x, IQR, median))

          
def categorize_production(x):
    m = int(x)
    k=''
    if m < df['production_budget'].quantile(0.33):
        k = 'small'
    if m > df['production_budget'].quantile(0.33) and x < df['production_budget'].quantile(0.66): 
        k = 'medium'
    if m > df['production_budget'].quantile(0.66): 
        k = 'large'
    return k

# df['budget_category'] = df['production_budget'].apply(lambda x: categorize_production(x))

In [19]:
find_outliers(df, 'roi').value_counts()

False    5139
True      643
Name: roi, dtype: int64

In [133]:
df.groupby(['budget_category'])['roi'].mean()

budget_category
          173.596193
large     161.911859
medium    212.265200
small     797.075949
Name: roi, dtype: float64